In [ ]:
import random
import math
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load("de")
spacy_en = spacy.load("en")

In [ ]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=(".de", ".en"),  fields=(SRC, TRG))

In [ ]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [ ]:
BATCH_SIZE = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE ,
    device = device
)

In [ ]:
class Encoder(nn.Module):

    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim*2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        # src -> [src_len, batch_size]
        embedding = self.dropout(self.embedding(src))
        # embedding = [src_len, batch_size, emb_dim]
        output, hidden = self.rnn(embedding)
        # output = [src_len, batch_size, hid_dim * num_directions]
        # hidden = [n_layers * num_directions, batch_size, hid_dim]
        fc = self.fc(  torch.cat(  (hidden[-2,:,:], hidden[-1,:,:]), dim=1  )  )
        hidden = torch.tanh(fc)
        # hidden = [batch_size, dec_hid_dim]
        return embedding, output, fc

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.attn = nn.Linear( (enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim )
        self.v = nn.Linear( dec_hid_dim, 1, bias=False )
    
    def forward(self, encoder_outputs, hidden):
        # hidden = [batch_size, dec_hid_dim]
        # encoder_outputs =  [src_len, batch_size, enc_hid_dim*2]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        # hidden = [batch_size, src_len, dec_hid_dim]
        # encoder_outputs = [batch_size, src_len, enc_hid_dim*2 ]

        energy = self.attn(  torch.cat( (hidden, encoder_outputs), dim=2 )  )
        energy = torch.tanh(energy)

        # energy = [batch_size, src_len, dec_hid_dim]

        attention = self.v(energy).squeeze(2)

        # attention = [batch_size, src_len]

        return F.softmax(attention, dim=1)




In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention ):
        super().__init__()
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU( (enc_hid_dim*2) + emb_dim, dec_hid_dim )
        self.fc_out = nn.Linear( (enc_hid_dim*2) + emb_dim + dec_hid_dim, output_dim )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inp, hidden, encoder_outputs):
        # inp = [batch_size]
        embedding = self.dropout( self.embedding( inp ) )
        # embedding = [1, batch_size, emb_dim]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch_size, 1, src_len]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch_size, src_len, enc_hid_dim*2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch_size, 1, enc_hid_dim*2]
        rnn_input = torch.cat( (embedded, weighted), dim=2 )
        # rnn_input = [1, batch_size, (enc_hid_dim*2) + emb_dim ]
        output, hidden = self.rnn( rnn_input, hidden.unsqueeze(0) )
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc( torch.cat( output, weighted, embedded ), dim=1 )

        return prediction, hidden.squeeze(0)


In [ ]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src = [src_len, batch_size]
        # trg = [trg_len, batch_size]

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        inp = trg[0,:]

        for t in range(1, trg_len):
            output, hidden = self.decoder(inp, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            inp = trg[t] if teacher_force else top1
        
        return outputs


In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("The model has {count_parameters(model):,} trainable parameters")

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG_vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PD_IDX)

In [ ]:
for 

    output = model(src, trg, 0)
    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)
    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()

return epoch_loss / len(iterator)
